In [ ]:
# !pip install symusic mido pretty_midi miditoolkit music21 matplotlib

In [ ]:
# !wget https://github.com/lzqlzzq/minimidi/raw/main/example/mahler.mid

# Load MIDI File

* print each objects in symusic could get the corresponding summaries
* Time unit is a quarter.
* A *pyi* file have been generated, you can browse the members and functions of each class in it.

In [7]:
from symusic import Score
score = Score("mahler.mid")
print("note_num: ", score.note_num())
print("start_time: ", score.start())
print("end_time: ", score.end())

note_num:  60411
start_time:  0
end_time:  1435615


In [ ]:
print(score.tempos)
print(score.key_signatures)
print(score.time_signatures)

In [ ]:
print(score.tempos[0])
print(score.key_signatures[0])
print(score.time_signatures[0])

In [ ]:
print("track name: ", score.tracks[0].name)
print("is_drum: ", score.tracks[0].is_drum)
print("note_num: ", score.tracks[0].note_num())
print("start: ", score.tracks[0].start())
print("end: ", score.tracks[0].end())
print("notes[0]: ", score.tracks[0].notes[0])
print("notes[-1]: ", score.tracks[0].notes[-1])

In [ ]:
note = score.tracks[0].notes[0]
print("start:\t\t", note.start)
print("duration:\t", note.duration)
print("pitch:\t\t", note.pitch)
print("velocity:\t", note.velocity)

# Batch Processing

* sort
* shift_time
* shift_pitch
* shift_velocity
* clip(start: float, end: float, clip_end: bool)
* filter_notes(func: Callable)
* note_array

In [ ]:
# inplace operation
print(score.sort())
print(score.tracks[0].sort())

In [ ]:
# method chaining
score.shift_time(10) \
     .shift_pitch(-6) \
     .shift_velocity(-7) \
     .sort()

In [ ]:
score.tracks[1].clip(1000, 100000, True) # start: float, end: float, clip_end: bool

In [ ]:
from matplotlib import pyplot as plt
score = score.resample(tpq=16, min_dur=1)
frame_pianoroll = score.tracks[3].pianoroll(modes=['frame'], pitch_range=(0, 128), encode_velocity=False)
print("frame_pianoroll", frame_pianoroll.dtype, frame_pianoroll.shape)
onset_pianoroll = score.tracks[1].pianoroll(modes=['onset'], pitch_range =(0, 128), encode_velocity=False)
print("onset_pianoroll", onset_pianoroll.dtype, onset_pianoroll.shape)
pianoroll = score.tracks[1].pianoroll(modes=['onset', 'frame'], pitch_range =(0, 128), encode_velocity=False)
print("both_pianoroll", pianoroll.dtype, pianoroll.shape)

total = score.pianoroll(modes=['frame']).max(axis=(0, 1)).astype(int)
plt.imshow(total, aspect='auto')

# Benchmark

## MIDI Parsing

* mido is writen in pure python, and only parsing midi file to event level
* pretty_midi and miditoolkit is based on mido

In [8]:
import mido, music21
import pretty_midi as pm
import miditoolkit as mtk
p = "mahler.mid"

In [9]:
%%timeit
Score(p)

3.22 ms ± 255 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [10]:
%%timeit
# mido is writen in pure python, and only parsing midi file to event level
mido.MidiFile(p)

1.48 s ± 18.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%%timeit
pm.PrettyMIDI(p)

In [ ]:
%%timeit
mtk.MidiFile(p)

In [ ]:
%%timeit
music21.converter.parse(p)

## Bench Processing


In [19]:
score = Score(p)
score2 = score.copy()
score_quant = score.resample(tpq=16, min_dur=1)

In [16]:
%%timeit
score.copy()

296 μs ± 23.5 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [17]:
%%timeit
score.shift_pitch(10)

452 μs ± 24.4 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [20]:
%%timeit
score_quant.pianoroll(modes=['frame', 'onset'], pitch_range=(0, 128), encode_velocity=True)

132 ms ± 8.34 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
